In [1]:
from gcsa.event import Event
from gcsa.google_calendar import GoogleCalendar
from gcsa.recurrence import Recurrence, DAILY, SU, SA

import plotly.express as px
import plotly.graph_objects as go

import pandas as pd

from datetime import datetime, timedelta, date
import dateutil
from dateutil.parser import parse
import calendar
import numpy as np
from circle_cal import Year_Data
import circle_cal as cc
import circle_cal.model as model

In [2]:
y=Year_Data(2024)
d_to_degree = 360 / len(y)

months = [date(y.year, i, 1) for i in range(1,13)] + [date(y.year+1, 1, 1)]
theta, width = cc.events_to_polar(months, days_to_theta = d_to_degree)
minor_ticks = [d_to_degree * y.date_as_number(date(y.year, m, 1)) for m in range(1,13)]


fig = go.Figure()
fig.add_trace(go.Barpolar(base=[.9] * 12, r=[.05] * 12,
                          theta=theta,
                          width=width,
                          marker_color="orange",
                          marker_line_color="black",
                          )
              )
tickvals = []
for u, v in zip(theta, minor_ticks):
    tickvals.append(u)
    tickvals.append(v)
ticktext = []
for m in calendar.month_abbr[1:]:
    ticktext.append(m)
    ticktext.append("")

fig.update_polars(
    dict(
        angularaxis=dict(
            rotation=-90,
            tickmode="array",
            tickvals=tickvals,
            ticktext=ticktext
        ),
    ),
    #margin = dict(t=0, l=0, r=0, b=0),
)
fig.update_layout(dict(height=1000))
fig.show()

TypeError: <circle_cal.model.Year_Data object at 0x10f066420> is used as a date but does not quack like one.

In [ ]:
datetime.now() in y

In [3]:
d_to_degree

NameError: name 'd_to_degree' is not defined

In [2]:
start = date.today()

In [3]:
end = date.today() + timedelta(days=100)

In [4]:
model._quacks_like_a_date(end)

True

In [6]:
model._date_or_dt(datetime.now())

'dt'

In [8]:
d = datetime.now()

In [10]:
model._quacks_like_a_dt(d)

True

In [16]:
type(d)

datetime.datetime

In [11]:
da=d.date()

In [ ]:
da.min